ਹੇਠਾਂ ਦਿੱਤੇ ਨੋਟਬੁੱਕ ਚਲਾਉਣ ਲਈ, ਜੇਕਰ ਤੁਸੀਂ ਹਜੇ ਤੱਕ ਨਹੀਂ ਕੀਤਾ, ਤਾਂ ਤੁਹਾਨੂੰ ਇੱਕ ਮਾਡਲ ਡਿਪਲੋਇ ਕਰਨ ਦੀ ਲੋੜ ਹੈ ਜੋ `text-embedding-ada-002` ਨੂੰ ਬੇਸ ਮਾਡਲ ਵਜੋਂ ਵਰਤਦਾ ਹੈ ਅਤੇ ਡਿਪਲੋਇਮੈਂਟ ਨਾਂ .env ਫਾਈਲ ਵਿੱਚ `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` ਵਜੋਂ ਸੈੱਟ ਕਰਨਾ ਹੈ।


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

ਅਗਲੇ ਕਦਮ ਵਿੱਚ, ਅਸੀਂ Embedding Index ਨੂੰ Pandas Dataframe ਵਿੱਚ ਲੋਡ ਕਰਨ ਜਾ ਰਹੇ ਹਾਂ। Embedding Index ਇੱਕ JSON ਫਾਈਲ `embedding_index_3m.json` ਵਿੱਚ ਸਟੋਰ ਕੀਤਾ ਗਿਆ ਹੈ। Embedding Index ਵਿੱਚ ਹਰ ਇੱਕ YouTube ਟ੍ਰਾਂਸਕ੍ਰਿਪਟ ਲਈ Embeddings ਹਨ, ਜੋ ਅਕਤੂਬਰ 2023 ਦੇ ਅੰਤ ਤੱਕ ਦੇ ਹਨ।


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

ਅਗਲੇ ਕਦਮ ਵਜੋਂ, ਅਸੀਂ ਇੱਕ ਫੰਕਸ਼ਨ ਬਣਾਉਣ ਜਾ ਰਹੇ ਹਾਂ ਜਿਸਦਾ ਨਾਮ `get_videos` ਹੋਵੇਗਾ, ਜੋ ਕਿ ਕੁਐਰੀ ਲਈ Embedding Index ਵਿੱਚ ਖੋਜ ਕਰੇਗਾ। ਇਹ ਫੰਕਸ਼ਨ ਉਹ ਪੰਜ ਵੀਡੀਓਜ਼ ਵਾਪਸ ਕਰੇਗਾ ਜੋ ਕੁਐਰੀ ਨਾਲ ਸਭ ਤੋਂ ਵੱਧ ਮਿਲਦੀਆਂ ਹਨ। ਇਹ ਫੰਕਸ਼ਨ ਹੇਠਾਂ ਦਿੱਤੇ ਤਰੀਕੇ ਨਾਲ ਕੰਮ ਕਰਦਾ ਹੈ:

1. ਸਭ ਤੋਂ ਪਹਿਲਾਂ, Embedding Index ਦੀ ਇੱਕ ਕਾਪੀ ਬਣਾਈ ਜਾਂਦੀ ਹੈ।
2. ਫਿਰ, ਕੁਐਰੀ ਲਈ Embedding OpenAI Embedding API ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਕਲਕੁਲੇਟ ਕੀਤਾ ਜਾਂਦਾ ਹੈ।
3. ਇਸ ਤੋਂ ਬਾਅਦ, Embedding Index ਵਿੱਚ ਇੱਕ ਨਵਾਂ ਕਾਲਮ ਬਣਾਇਆ ਜਾਂਦਾ ਹੈ ਜਿਸਦਾ ਨਾਮ `similarity` ਹੈ। `similarity` ਕਾਲਮ ਵਿੱਚ ਕੁਐਰੀ Embedding ਅਤੇ ਹਰ ਵੀਡੀਓ ਸੈਗਮੈਂਟ ਦੀ Embedding ਵਿਚਕਾਰ cosine similarity ਹੁੰਦੀ ਹੈ।
4. ਫਿਰ, Embedding Index ਨੂੰ `similarity` ਕਾਲਮ ਰਾਹੀਂ ਫਿਲਟਰ ਕੀਤਾ ਜਾਂਦਾ ਹੈ। Embedding Index ਨੂੰ ਇਸ ਤਰ੍ਹਾਂ ਫਿਲਟਰ ਕੀਤਾ ਜਾਂਦਾ ਹੈ ਕਿ ਸਿਰਫ ਉਹ ਵੀਡੀਓਜ਼ ਹੀ ਸ਼ਾਮਲ ਹੋਣ ਜਿਨ੍ਹਾਂ ਦੀ cosine similarity 0.75 ਜਾਂ ਇਸ ਤੋਂ ਵੱਧ ਹੋਵੇ।
5. ਆਖ਼ਰ ਵਿੱਚ, Embedding Index ਨੂੰ `similarity` ਕਾਲਮ ਮੁਤਾਬਕ sort ਕੀਤਾ ਜਾਂਦਾ ਹੈ ਅਤੇ ਸਭ ਤੋਂ ਉੱਤੇ ਵਾਲੀਆਂ 5 ਵੀਡੀਓਜ਼ ਵਾਪਸ ਕਰ ਦਿੱਤੀਆਂ ਜਾਂਦੀਆਂ ਹਨ।


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ਇਹ ਫੰਕਸ਼ਨ ਬਹੁਤ ਸਧਾਰਣ ਹੈ, ਇਹ ਸਿਰਫ਼ ਖੋਜ ਪ੍ਰਸ਼ਨ ਦੇ ਨਤੀਜੇ ਪ੍ਰਿੰਟ ਕਰਦਾ ਹੈ।


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ਸਭ ਤੋਂ ਪਹਿਲਾਂ, Embedding Index ਨੂੰ Pandas Dataframe ਵਿੱਚ ਲੋਡ ਕੀਤਾ ਜਾਂਦਾ ਹੈ।
2. ਅਗਲੇ ਕਦਮ 'ਚ, ਯੂਜ਼ਰ ਨੂੰ ਇੱਕ ਕੁਇਰੀ ਦਰਜ ਕਰਨ ਲਈ ਕਿਹਾ ਜਾਂਦਾ ਹੈ।
3. ਫਿਰ `get_videos` ਫੰਕਸ਼ਨ ਨੂੰ ਕਾਲ ਕਰਕੇ Embedding Index ਵਿੱਚੋਂ ਕੁਇਰੀ ਲਈ ਖੋਜ ਕੀਤੀ ਜਾਂਦੀ ਹੈ।
4. ਆਖਰ ਵਿੱਚ, `display_results` ਫੰਕਸ਼ਨ ਨੂੰ ਕਾਲ ਕਰਕੇ ਨਤੀਜੇ ਯੂਜ਼ਰ ਨੂੰ ਵਿਖਾਏ ਜਾਂਦੇ ਹਨ।
5. ਫਿਰ ਯੂਜ਼ਰ ਨੂੰ ਇੱਕ ਹੋਰ ਕੁਇਰੀ ਦਰਜ ਕਰਨ ਲਈ ਕਿਹਾ ਜਾਂਦਾ ਹੈ। ਇਹ ਪ੍ਰਕਿਰਿਆ ਤਦ ਤੱਕ ਚੱਲਦੀ ਰਹਿੰਦੀ ਹੈ ਜਦ ਤੱਕ ਯੂਜ਼ਰ `exit` ਨਹੀਂ ਲਿਖਦਾ।

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.pa.png)

ਤੁਹਾਨੂੰ ਇੱਕ ਕੁਇਰੀ ਦਰਜ ਕਰਨ ਲਈ ਕਿਹਾ ਜਾਵੇਗਾ। ਆਪਣੀ ਕੁਇਰੀ ਲਿਖੋ ਅਤੇ ਐਂਟਰ ਦਬਾਓ। ਐਪਲੀਕੇਸ਼ਨ ਤੁਹਾਡੀ ਕੁਇਰੀ ਨਾਲ ਸੰਬੰਧਤ ਵੀਡੀਓਜ਼ ਦੀ ਲਿਸਟ ਵਾਪਸ ਕਰੇਗੀ। ਐਪਲੀਕੇਸ਼ਨ ਤੁਹਾਨੂੰ ਵੀਡੀਓ ਵਿੱਚ ਉਸ ਥਾਂ ਦਾ ਲਿੰਕ ਵੀ ਦੇਵੇਗੀ ਜਿੱਥੇ ਸਵਾਲ ਦਾ ਜਵਾਬ ਮਿਲਦਾ ਹੈ।

ਇਹ ਕੁਝ ਉਦਾਹਰਨੀ ਕੁਇਰੀਆਂ ਹਨ, ਜੋ ਤੁਸੀਂ ਅਜ਼ਮਾਈਆਂ:

- Azure Machine Learning ਕੀ ਹੈ?
- Convolutional neural networks ਕਿਵੇਂ ਕੰਮ ਕਰਦੀਆਂ ਹਨ?
- Neural network ਕੀ ਹੈ?
- ਕੀ ਮੈਂ Jupyter Notebooks ਨੂੰ Azure Machine Learning ਨਾਲ ਵਰਤ ਸਕਦਾ ਹਾਂ?
- ONNX ਕੀ ਹੈ?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**ਅਸਵੀਕਰਨ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਅਸੀਂ ਯਥਾਸੰਭਵ ਸਹੀ ਅਨੁਵਾਦ ਕਰਨ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਦੇ ਹਾਂ, ਪਰ ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਵਿੱਚ ਰੱਖੋ ਕਿ ਆਟੋਮੈਟਿਕ ਅਨੁਵਾਦ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਣਪੂਰੀ ਜਾਣਕਾਰੀ ਹੋ ਸਕਦੀ ਹੈ। ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਅਸਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਹੀ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫ਼ਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਹੋਣ ਵਾਲੀਆਂ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
